In [ ]:
import wandb
from wandb.keras import WandbCallback
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/datadir-celeb-a/aligned_celeba.txt', sep='\t')
train_df, test_df = train_test_split(df, test_size=0.2)
train_df['datadir'] = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/' + train_df['datadir'].astype(str)
test_df['datadir'] = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/' + test_df['datadir'].astype(str)

In [ ]:
defaults = {
    'epochs': 20,
    'batch_size': 128,
    'fc1_num_neurons': 1024,
    'fc2_num_neurons': 512,
    'fc3_num_neurons': 256,
    'seed': 7,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
}

wandb.init(config=defaults, resume=True, name='First Model', project='CelebA Runs')
config = wandb.config

# Load images into keras image generator 
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)
datagen_test = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

test_generator = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

mobile_net_v2 = tf.keras.applications.MobileNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(224,224,3),
)
mobile_net_v2.trainable = True

fc1 = tf.keras.layers.Dense(
    config.fc1_num_neurons,
    activation=config.hidden_activation,
)

fc2 = tf.keras.layers.Dense(
    config.fc2_num_neurons,
    activation=config.hidden_activation,
)

fc3 = tf.keras.layers.Dense(
    config.fc2_num_neurons,
    activation=config.hidden_activation,
)

model = tf.keras.models.Sequential([
    mobile_net_v2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    fc1,
    tf.keras.layers.BatchNormalization(),
    fc2,
    tf.keras.layers.BatchNormalization(),
    fc3,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation=config.output_activation),
])

model.summary()

# Compile model 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=config.loss_function,
    metrics=config.metrics,
)

model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=config.epochs,
    callbacks=[WandbCallback()],
)
model.save_weights('model_celeba.h5') 
run.finish()